### Network stability analysis
* done using the abundance-weighted mean interaction strength (wMISi) index
* calculate stability using sum of wMISi indices

In [78]:
# import statements

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle
from math import comb
import statistics
import glob
from tqdm import tqdm

dis = "crc"

files = glob.glob('../results/'+dis+'/'+dis+'_dicts/*')
# ibd_files = glob.glob('../results/'+ibd+'/'+ibd+'_dicts/*')
# crc_files = glob.glob('../results/'+ibs+'/'+ibs+'_dicts/*')

graph = pickle.load(open("../results/graphs/"+dis+"_graph.pkl", "rb"))
# ibd_graph = pickle.load(open("../results/graphs/ibd_graph.pkl", "rb"))
# crc_graph = pickle.load(open("../results/graphs/crc_graph.pkl", "rb"))

classifier = "family"

In [79]:
# calculate relative abundance (tot):
tot = {}
num = 0
denom = 0

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
    

    # count total taxa
    for i in taxa_dict[classifier]:
        if i not in tot:
            tot[i] = taxa_dict[classifier][i]
        else:
            tot[i] += taxa_dict[classifier][i]
        denom += taxa_dict[classifier][i]

for i in tot:
    tot[i] /= denom 
    tot[i] *= 100

In [82]:
def cov(x, y):
    res = 0
    for file in files:
        with open(file, 'rb') as f:
            taxa_dict = pickle.load(f)
        if x in taxa_dict[classifier] and y in taxa_dict[classifier]:
            res += (taxa_dict[classifier][x] - (tot[x] / len(files))) * (taxa_dict[classifier][y] - (tot[y] / len(ibs_files))) 

    res /= (len(files)-1)
    return res

def std(x):
    res = 0
    for file in files:
        with open(file, 'rb') as f:
            taxa_dict = pickle.load(f)
        if x in taxa_dict[classifier]:
            res += (taxa_dict[classifier][x] - (tot[x] / len(files)))**2

    res /= (len(files))
    res = res**(1/2)
    return res

def spearman(x, y):
    return (cov(x, y)/(std(x) * std(y)))

In [83]:
mis_dict = {}
for node in tqdm(graph.nodes):
    num = 0
    for j in graph.neighbors(node):
        num += tot[j] * spearman(node, j)
        denom += tot[j]
    mis_dict[node] = num / denom

print(mis_dict)

100%|██████████| 614/614 [13:25<00:00,  1.31s/it]

{'Weeksellaceae': 0.0, 'Odoribacteraceae': 0.0, 'Fusobacteriaceae': 0.0, 'Lachnospiraceae': 0.0, 'Prevotellaceae': 0.0, 'Peptoniphilaceae': 0.0, 'Bacteroidaceae': 0.0, 'Corynebacteriaceae': 0.0, 'Micrococcaceae': 0.0, 'Veillonellaceae': 0.0, 'Coriobacteriaceae': 0.0, 'Microbacteriaceae': 0.0, 'Petrotogaceae': 0.0, 'Erysipelotrichaceae': 0.0, 'Balneicellaceae': 0.0, 'Moraxellaceae': 0.0, 'Erwiniaceae': 0.0, 'Morganellaceae': 0.0, 'Gracilibacteraceae': 7.707087771638611e-06, 'Bacillaceae': 0.0, 'Eubacteriales_incertae_sedis_XIII': 0.0, 'Burkholderiaceae': 0.0, 'Porphyromonadaceae': 0.0, 'Pseudomonadaceae': 6.396008151509125e-06, 'Synergistaceae': 0.0, 'Streptococcaceae': 0.0, 'Williamwhitmaniaceae': 0.0, 'Propionibacteriaceae': 0.0, 'XI': 0.0, '1': 0.0, 'Enterococcaceae': 0.0, 'Acholeplasmataceae': 0.0, 'Marinimicrobia_genera_incertae_sedis': 0.0, 'Peptostreptococcaceae': 0.0, 'Enterobacteriaceae': 0.0, 'Mesoaciditogaceae': 1.127927670613178e-07, 'Methanomassiliicoccaceae': 0.0, 'Victiva

In [84]:
deg_list = []
deg_list_nonzero = []

for node in graph.nodes:
    deg = len(graph.edges(node))
    deg_list.append(deg)
    if deg > 0:
        deg_list_nonzero.append(deg)

mean_deg = statistics.mean(deg_list)

imp = []
for node in tqdm(graph.nodes):
    if len(graph.edges(node)) > 2 * mean_deg:
        imp.append(node)

num = 0
denom = 0
for i in imp:
    num += mis_dict[i]

for i in graph.nodes:
    denom += mis_dict[i]

print(num/ denom)

100%|██████████| 614/614 [00:00<00:00, 111799.55it/s]

0.6547715951763016


In [77]:
with open("../results/"+dis+"_mis_dict.pkl", 'wb') as f:
    pickle.dump(mis_dict, f)